In [1]:
import cv2
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import Tracker

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = YOLO("yolov9c.pt")
tracker = Tracker()

target_classes = {2, 7}
cap = cv2.VideoCapture("5473765-uhd_3840_2160_24fps.mp4")
count_vehicles = 0
counter_ids = set()


frame_width = int(cap.get(3))  
frame_height = int(cap.get(4)) 
fps = int(cap.get(cv2.CAP_PROP_FPS))  


fourcc = cv2.VideoWriter_fourcc(*"mp4v") 
out = cv2.VideoWriter(
    "output.mp4", fourcc, fps, (640, 640)
) 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 640))
    frame_tensor = torch.from_numpy(frame_resized.astype(np.float32)).to(device)
    frame_tensor = frame_tensor.permute(2, 0, 1).unsqueeze(0) / 255.0

    results = model.predict(frame_tensor)
    a = results[0].boxes.data.to("cpu").numpy()
    px = pd.DataFrame(a).astype("float")

    detected_objects = []
    for _, row in px.iterrows():
        x1, y1, x2, y2 = map(int, row[:4])
        class_id = int(row[5])

        if class_id in target_classes:
            detected_objects.append([x1, y1, x2 - x1, y2 - y1])

    bbox_id = tracker.update(detected_objects)

    y_line = 550
    offset = 7

    for bbox in bbox_id:
        x3, y3, w, h, obj_id = bbox
        cx = x3 + w // 2
        cy = y3 + h // 2

        cv2.circle(frame_resized, (cx, cy), 4, (0, 0, 255), -1)
        cv2.rectangle(frame_resized, (x3, y3), (x3 + w, y3 + h), (0, 255, 0), 2)

        if y_line - offset < cy < y_line + offset:
            if obj_id not in counter_ids:
                counter_ids.add(obj_id)
                count_vehicles += 1

    cv2.putText(
        frame_resized,
        f"Vehicles: {count_vehicles}",
        (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA,
    )

    cv2.line(frame_resized, (20, y_line), (640, y_line), (0, 0, 255), 3)

    out.write(frame_resized)

    frame_display = cv2.resize(frame_resized, (640, 360))
    cv2.imshow("Vehicle Counter", frame_display)

    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
out.release()  
cv2.destroyAllWindows()

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Using device: cuda


100%|██████████| 49.4M/49.4M [01:07<00:00, 762kB/s] 



0: 640x640 9 cars, 5 trucks, 76.9ms
Speed: 1.8ms preprocess, 76.9ms inference, 239.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 cars, 5 trucks, 66.2ms
Speed: 0.0ms preprocess, 66.2ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 cars, 1 bus, 3 trucks, 67.1ms
Speed: 1.0ms preprocess, 67.1ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 cars, 1 bus, 5 trucks, 66.6ms
Speed: 0.0ms preprocess, 66.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 cars, 4 trucks, 57.5ms
Speed: 0.0ms preprocess, 57.5ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 cars, 3 trucks, 58.2ms
Speed: 0.0ms preprocess, 58.2ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 3 trucks, 57.1ms
Speed: 0.0ms preprocess, 57.1ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 cars, 1 bus, 7 trucks, 57.2ms
Sp